In [ ]:
%pip install sklearn pandas matplotlib seaborn 

In [3]:
%pip install scikit-learn


  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/2d/30/3afb8bcb785653254eb646ff2680ec4d637b40b06f4b046aca17b5e086b0/scikit_learn-1.3.0-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/e9/20/2d0561ab54d857365926c5b53538369a7b8d6ccbffaca509305b074028cd/scipy-1.11.2-cp39-cp39-win_amd64.whl.metadata
     ---------------------------------------- 0.0/59.1 kB ? eta -:--:--
     ------ --------------------------------- 10.2/59.1 kB ? eta -:--:--
     ------------------- ------------------ 30.7/59.1 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 59.1/59.1 kB 626.5 kB/s eta 0:00:00
  Obtaining dependency information for joblib>=1.1.1 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from 

In [4]:
import pandas as pd
import numpy as np

import seaborn as sns
sns.set(style="white", color_codes=True)
sns.set(font_scale=1.5)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# import plotting libraries
import matplotlib.pyplot as plt

1...50: 
      Average, standard deviation, min, max and median of the 
      Attributes 51...60 for the source of the current blog post
      With source we mean the blog on which the post appeared. 
      For example, myblog.blog.org would be the source of 
      the post myblog.blog.org/post_2010_09_10 
      
51:   Total number of comments before basetime

52:   Number of comments in the last 24 hours before the 
      basetime
      
53:   Let T1 denote the datetime 48 hours before basetime,
      Let T2 denote the datetime 24 hours before basetime.
      This attribute is the number of comments in the time period 
      between T1 and T2
      
54:   Number of comments in the first 24 hours after the 
      publication of the blog post, but before basetime
      
55:   The difference of Attribute 52 and Attribute 53

56...60: 
      The same features as the attributes 51...55, but  
      features 56...60 refer to the number of links (trackbacks), 
      while features 51...55 refer to the number of comments.
      
61:   The length of time between the publication of the blog post 
      and basetime
      
62:   The length of the blog post

**63...262: 
      The 200 bag of words features for 200 frequent words of the 
      text of the blog post**
      
263...269: binary indicator features (0 or 1) for the weekday
      (Monday...Sunday) of the basetime
      
270...276: binary indicator features (0 or 1) for the weekday
      (Monday...Sunday) of the date of publication of the blog
      post
      
277:  Number of parent pages: we consider a blog post P as a
      parent of blog post B, if B is a reply (trackback) to 
      blog post P.
      
278...280:  
      Minimum, maximum, average number of comments that the 
      parents received
      
281:  The target: the number of comments in the next 24 hours
      (relative to basetime)

In [5]:
# Load the dataset
data_url = "https://raw.githubusercontent.com/gridflowai/gridflowAI-datasets-icons/master/AI-DATASETS/01-MISC/blogfeedback/blogData_train.csv"
df = pd.read_csv(data_url, header=None)

In [6]:
df.shape

(52397, 281)

In [7]:
df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,271,272,273,274,275,276,277,278,279,280
34098,3.046667,6.444984,0.0,44.0,1.5,1.333333,3.501746,0.0,29.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16884,1.675439,8.113229,0.0,74.0,0.0,0.622807,4.374749,0.0,65.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46780,84.750610,91.961070,0.0,510.0,58.0,32.814182,57.297016,0.0,368.0,5.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
45378,10.146341,32.393284,0.0,292.0,0.0,5.151219,20.454865,0.0,220.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3738,29.267677,19.170923,0.0,122.0,26.0,11.098485,13.550798,0.0,80.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [8]:
df.values[:10, 65:190]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
df.dtypes

0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
276    float64
277    float64
278    float64
279    float64
280    float64
Length: 281, dtype: object

#### check null values

In [10]:
df.isnull().sum()

0      0
1      0
2      0
3      0
4      0
      ..
276    0
277    0
278    0
279    0
280    0
Length: 281, dtype: int64

In [11]:
NAs = pd.concat([df.isnull().sum()], axis=1, keys=['Count of NANs'])
NAs.sample(5)

,Count of NANs
93,0
126,0
230,0
213,0
204,0


In [12]:
NAs[NAs.sum(axis=1) > 0]

,Count of NANs


In [13]:
y = df.iloc[:, -1]

#### Distribution of y - histogram, normality test, outliers

In [14]:
df.iloc[:, -1].describe()

count    52397.000000
mean         6.764719
std         37.706565
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max       1424.000000
Name: 280, dtype: float64

#### type of columns
let us have a look at the data type of all the variables present in the dataset.

In [15]:
dtype_df = df.dtypes.reset_index()
dtype_df.sample(10)

,index,0
187,187,float64
15,15,float64
108,108,float64
207,207,float64
132,132,float64
219,219,float64
141,141,float64
31,31,float64
67,67,float64
114,114,float64


In [16]:
dtype_df.columns = ["Count", "Column Type"]
dtype_df.head(3)

,Count,Column Type
0,0,float64
1,1,float64
2,2,float64


In [17]:
dtype_df.groupby("Column Type").aggregate('count').reset_index()

,Column Type,Count
0,float64,281


In [18]:
# Split the data into features (X) and target (y)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [20]:
def model_metrics(y_true, y_pred, test='test data'): 
    print("metrics using {} ...".format(test))
    print('Mean Abs Error   MAE    : ', mean_absolute_error(y_true, y_pred))
    print('Mean Sq  Error MSE      : ', mean_squared_error(y_true, y_pred))
    print('Root Mean Sq Error RMSE : ', np.sqrt(mean_squared_error(y_true, y_pred)))
    # print('MAPE                    : ', np.mean(np.abs((y_true - y_pred) / y_true)) * 100)
    # print('MPE                     : ', np.mean((y_true - y_pred) / y_true) * 100)
    print('r2 value                : ', r2_score(y_true, y_pred))

    print('\n')

In [21]:
np.set_printoptions(precision=4, suppress=True)

In [22]:
# Replace zero values with 0.000001
y_train_zero_inflated = np.where(y_train == 0, 0.000001, y_train)
y_test_zero_inflated  = np.where(y_test  == 0, 0.000001, y_test)

In [24]:
knn_regressor = KNeighborsRegressor(
    n_neighbors=5,
    weights  ='distance',
    algorithm='auto',
    leaf_size=30,
    metric   ='hamming',
    p        =1,
    n_jobs   =-1
)

knn_regressor.fit(X_train, y_train_zero_inflated)

KNeighborsRegressor(metric='hamming', n_jobs=-1, p=1, weights='distance')

In [25]:
# predict (training samples)
y_pred = knn_regressor.predict(X_train)
model_metrics(y_train_zero_inflated, y_pred, 'training data')

# predict (testing samples)
y_pred = knn_regressor.predict(X_test)
model_metrics(y_test_zero_inflated, y_pred, 'test data')

metrics using training data ...
Mean Abs Error   MAE    :  0.01919268526134981
Mean Sq  Error MSE      :  0.07363192797290147
Root Mean Sq Error RMSE :  0.2713520369794586
r2 value                :  0.9999475931058427


metrics using test data ...
Mean Abs Error   MAE    :  6.514596733738967
Mean Sq  Error MSE      :  1025.9663108681095
Root Mean Sq Error RMSE :  32.030708872394776
r2 value                :  0.3108551781937423




In [26]:
from sklearn.model_selection import RandomizedSearchCV

In [27]:
# Create a dictionary of parameter distributions to search over
param_dist = {
    'n_neighbors': np.arange(1, 21),  # Range of values for n_neighbors
    #'weights': ['uniform', 'distance'],
    'algorithm': ['ball_tree', 'kd_tree', 'brute'],
    #'leaf_size': np.arange(10, 51),
    'p': [1, 2,3,4,5]
}

In [28]:
# Create a KNeighborsRegressor instance
knn_regressor = KNeighborsRegressor()

# Create a RandomizedSearchCV instance
random_search = RandomizedSearchCV(
    knn_regressor,
    param_distributions=param_dist,
    n_iter=100,                        # Number of parameter settings to sample
    scoring='neg_mean_squared_error',  # Use a suitable scoring metric
    cv=5,                              # Number of cross-validation folds
    n_jobs=-1                          # Use all available CPU cores
)

In [ ]:
# Fit the RandomizedSearchCV instance to your data
random_search.fit(X_train, y_train_zero_inflated)  # Replace X_train and y_train with your data

# Print the best parameters and the corresponding score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", -random_search.best_score_)  # Convert negative MSE back to positive

# Get the best KNeighborsRegressor model
best_knn_regressor = random_search.best_estimator_

In [ ]:
model_metrics(y_true,random_search.best_estimator_.predict(X_test))

#### try XGBOOST

In [ ]:
import xgboost as xgb

In [ ]:
# Create an XGBoost regressor instance
regressor = xgb.XGBRegressor(
    n_estimators  = 100,       # Number of boosting rounds (trees)
    max_depth     = 14,        # Maximum depth of each tree
    learning_rate = 0.075,      # Step size at each iteration
    objective     = 'reg:squarederror',  # Specify regression task
    #random_state  = 42,
    colsample_bytree = .75,
    reg_alpha        = .00001,                 # L1 regularization
    reg_lambda       = 10,
    eval_metric      = 'mae',
    subsample        = .75, 
)

In [ ]:
# Train the regressor on the training data
regressor.fit(X_train, y_train_zero_inflated)

In [ ]:
# predict (training samples)
y_pred = regressor.predict(X_train)
model_metrics(y_train_zero_inflated, y_pred, 'training data')

# predict (testing samples)
y_pred = regressor.predict(X_test)
model_metrics(y_test_zero_inflated, y_pred, 'test data')

#### K-Fold with XGBOOST

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
# Define the number of folds
num_folds = 5

# Initialize the KFold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

In [ ]:
# Initialize a list to store the mean squared errors for each fold
my_train_zero_inflateds = []
mse_scores = []
r2_score   = []

# Create an XGBoost regressor instance
regressor = xgb.XGBRegressor(
    n_estimators  = 100,       # Number of boosting rounds (trees)
    max_depth     = 14,        # Maximum depth of each tree
    learning_rate = 0.075,      # Step size at each iteration
    objective     = 'reg:squarederror',  # Specify regression task
    #random_state  = 42,
    colsample_bytree = .75,
    reg_alpha        = .01,                 # L1 regularization
    reg_lambda       = .01,
    eval_metric      = 'mae',
    subsample        = .75, 
)

# Perform k-fold cross-validation using cross_val_score
num_folds = 5
r2_scores = cross_val_score(regressor, X_train, y_train_zero_inflated, cv=num_folds, scoring='r2')

In [ ]:
r2_scores

#### try lightGBM

In [ ]:
#!pip install lightgbm

In [ ]:
import lightgbm as lgb

In [ ]:
# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train_zero_inflated)
test_data  = lgb.Dataset(X_test,  label=y_test_zero_inflated, reference=train_data)

In [ ]:
# Define hyperparameters for the LightGBM model
params = {
    'objective': 'regression',  # This specifies that we're solving a regression problem
    'metric': 'rmse',           # Root Mean Squared Error
    'boosting_type': 'gbdt',    # Gradient Boosting Decision Tree
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the LightGBM model
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# Make predictions on the test set
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_zero_inflated, y_pred))
print(f"RMSE: {rmse}")

print('r2 value                : ', r2_score(y_test_zero_inflated, y_pred))

In [ ]:
import sklearn.metrics

In [ ]:
sklearn.metrics.r2_score(y_test_zero_inflated, y_pred)